Wave Equation
===

We solve the first order wave equation by a matrix-free explicit DG method.

\begin{eqnarray*}
\frac{\partial p}{\partial t} & = & \operatorname{div} u \\
\frac{\partial u}{\partial t} & = & \nabla p
\end{eqnarray*}

We obtain the ODE 
\begin{eqnarray*}
M_p \dot{p} & = & -B^T u \\
M_u \dot{u} & = & B p
\end{eqnarray*}

form a simple DG version with central fluxes. The discrete gradient $B$ is defined by the bilinear-form
$$
b(p,v) = \sum_{T}
\Big\{ \int_T \nabla p  \, v + \int_{\partial T} (\{ p \} - p) \, v_n \, ds \Big\} 
$$

It conserves energy, but is not free of spurious modes.

Hesthaven+Warbuton: Nodal Discontinuous Galerkin Methods

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
from ngsolve.krylovspace import CGSolver
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

### Different geometries are possible (1line, 2line, hole, box 3d)

In [ ]:
def create_mesh(geo, dim, extrude):
    if geo == "hole":
        box = MoveTo(-1,-1).Rectangle(2,2).Face()
        implicit = MoveTo(0.1,-0.005).Rectangle(0.3,0.005).Face() + \
            MoveTo(0.2,-0.15).Rectangle(0.005, 0.3).Face() 
        circ = Circle((0.5, 0), r = 0.15).Face()
        circ.maxh = h/4
        shape = box - circ - implicit
    elif geo == "1line":
        rect = MoveTo(-1,-1).Rectangle(2.5,2).Face()
        hole = MoveTo(0.5,0.01).Rectangle(0.001,0.8).Face()
        shape = rect-hole
    elif geo == "2line":
        rect = MoveTo(-0.5,-0.5).Rectangle(1.5,1).Face()
        hole = MoveTo(0.5,0.01).Rectangle(0.001,0.4).Face() + \
            MoveTo(0.5,-0.41).Rectangle(0.001,0.4).Face()
        shape = rect-hole
    elif geo == "box": 
        box = Box((-1,-1,-1), (1,1,0))
        hole = Sphere( Pnt(0.5, 0, 0), 0.2 )
        shape = box - hole
        shape.faces.name = "outer"
        return shape
    
    if dim == 3:
        shape = shape.Extrude(extrude)
    return shape

In [ ]:
dim = 2
extrude = 0.05
h = 0.05
geo = "2line" # hole, 1line, 2line, box

shape = create_mesh(geo, dim, extrude)

shape = OCCGeometry(shape, dim = dim)

mesh = Mesh(shape.GenerateMesh(maxh = h)).Curve(3)
Draw(mesh)

In [ ]:
print("boundaries: ", mesh.GetBoundaries())
print("materials: ", mesh.GetMaterials())
print("number elements: ", mesh.ne)

In [ ]:
order = 3
fes_p = L2(mesh, order=order+1, all_dofs_together=True, dgjumps=True) 
fes_u = VectorL2(mesh, order=order, piola=True, dgjumps=True)
fes_tr = FacetFESpace(mesh, order=order+1)
fes = fes_p * fes_u

traceop = fes_p.TraceOperator(fes_tr, average=True) 

gfu = GridFunction(fes_u)
gfp = GridFunction(fes_p)
gftr = GridFunction(fes_tr)

gfp.Set( exp(-400*(x**2+y**2+z**2)))
gftr.vec.data = traceop * gfp.vec

p, u = fes.TrialFunction()
q, v = fes.TestFunction()
phat = fes_tr.TrialFunction()

n = specialcf.normal(mesh.dim)
dS = dx(skeleton=True)   
def jump(p): return p.Other()-p
def avgn(v): return 0.5*(v*n-v.Other()*n.Other())

In [ ]:
embp, embu = fes.embeddings

In [ ]:
tend = 2
dt = 0.2 * h / (order+1)**2
print ("dt = ", dt)
print("Number of steps: ", tend/dt)

In [ ]:
print("fes ndofs: ", fes.ndof)
print("fes_u ndofs: ", fes_u.ndof)
print("fes_p ndofs: ", fes_p.ndof)
print("fes_tr ndofs: ", fes_tr.ndof)

In [ ]:
if dim == 2:
    Draw (gfp, order=3)
else:
    gftr.vec.data = traceop * gfp.vec
    Draw (gftr, draw_vol=False, order=3);

In [ ]:
mesh.ne

In [ ]:
F = specialcf.JacobianMatrix(mesh.dim)
Finv = Inv(F)
detF = Det(F)
Norm_Finv = Norm(Finv)
el_norms = Integrate(Norm_Finv*1/detF.Norm(), mesh, element_wise=True)
el_norms_numpy = np.array(el_norms)

sorted_el_norms = -np.sort(-el_norms_numpy) #sort descending
ref_norm = el_norms_numpy.mean()# using elements > mean 
impl_els = np.where(el_norms_numpy > ref_norm, 1, 0)
    
print('max', max(el_norms_numpy))
print('min', min(el_norms_numpy))
print('mean', el_norms_numpy.mean())
print('median', np.median(el_norms_numpy))
print('ref_norm', ref_norm)

In [ ]:
ba_implicit_els = BitArray(mesh.ne)
ba_explicit_els = BitArray(mesh.ne)
ba_interface_edges = BitArray(mesh.nedge)
ba_explicit_edges = BitArray(mesh.nedge)
ba_implicit_edges = BitArray(mesh.nedge)

ba_implicit_els[:] = 0
ba_explicit_els[:] = 0
ba_interface_edges[:] = 0
ba_explicit_edges[:] = 0
ba_implicit_edges[:] = 0

for el in mesh.Elements():
    if impl_els[el.nr] == 1:
        ba_implicit_els[el.nr] = 1
        for e in el.edges:
            ba_implicit_edges[e.nr] = 1
    else:
        ba_explicit_els[el.nr] = 1
        for e in el.edges:
            ba_explicit_edges[e.nr] = 1
        
ba_interface_edges = ba_explicit_edges & ba_implicit_edges     

In [ ]:
ba_local_implicit_dofs = BitArray(fes.ndof)
ba_local_implicit_dofs[:] = 0

for el in mesh.Elements():
    if ba_implicit_els[el.nr] == 1:
        for nr in fes.GetDofNrs(el):
            ba_local_implicit_dofs[nr] = 1
    if ba_explicit_els[el.nr] == 1:
        for e in el.edges:
            if ba_interface_edges[e.nr] == 1:
                for nr in fes.GetDofNrs(el):
                    ba_local_implicit_dofs[nr] = 1

ba_local_implicit_dofs_u = BitArray(fes_u.ndof)
ba_local_implicit_dofs_u[:] = 0

for el in mesh.Elements():
    if ba_implicit_els[el.nr] == 1:
        for nr in fes_u.GetDofNrs(el):
            ba_local_implicit_dofs_u[nr] = 1
    if ba_explicit_els[el.nr] == 1:
        for e in el.edges:
            if ba_interface_edges[e.nr] == 1:
                for nr in fes_u.GetDofNrs(el):
                    ba_local_implicit_dofs_u[nr] = 1

In [ ]:
A = BilinearForm(fes)
A += -p*q*dx +u*v*dx 
A += dt/2*(grad(p)*v + grad(q)*u)*dx
A += dt/2*(jump(p)*avgn(v)+jump(q)*avgn(u)) * dS
A.Assemble()

p1, q1 = fes_p.TnT()
u1, v1 = fes_u.TnT()
phat1, qhat1 = fes_tr.TnT()
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = True)
Bel += grad(p1)*v1 * dx -p1*(v1*n) * dx(element_boundary=True)
Bel.Assemble()
Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = True)
Btr += phat1 * (v1*n) *dx(element_boundary=True)
Btr.Assemble();

B = Bel.mat + Btr.mat @ traceop
B_T = B.T

In [ ]:
massu = fes_u.Mass(1)
invmassu = fes_u.Mass(1).Inverse()
invmassp = fes_p.Mass(1).Inverse()

In [ ]:
Ps = Projector(ba_local_implicit_dofs_u, True)   # projection to small
Pl = Projector(ba_local_implicit_dofs_u, False)  # projection to large
B_e = Pl @ B
B_i = Ps @ B

print ("local implicit dofs: ", ba_local_implicit_dofs.NumSet(),"/",len(ba_local_implicit_dofs))
print ("local implicit dofs of fes u: ", ba_local_implicit_dofs_u.NumSet(),"/",len(ba_local_implicit_dofs_u))
print ("shape B_e: ", B_e.shape)
print ("shape B_i: ", B_i.shape)
print ("shape Ps: ", Ps.shape)

In [ ]:
invA = A.mat.Inverse(freedofs=ba_local_implicit_dofs, inverse="sparsecholesky")

# delete non zeros elements of matrix to speed up matrix multiplication
Anze = A.mat.DeleteZeroElements(10e-12)
invAnze = Anze.Inverse(freedofs=ba_local_implicit_dofs, inverse="sparsecholesky")
invmstar = embu.T @ invAnze @ embu
mstarloc = massu + dt*dt/4*B_i @ invmassp @ B_T

In [ ]:
A.mat.nze, invA.nze, Anze.nze

In [ ]:
invmassuB = invmassu @ B
invmasspB_T = invmassp @ B_T
invmstar_mstar = invmstar @ mstarloc

In [ ]:
# visulaize implicit dofs
from time import sleep
ba_gfu = BitArray(fes.ndof)
ba_gfu[:] = 0
gfuim = GridFunction(fes)
gfuim.vec[:] = 0
for i in range(len(ba_local_implicit_dofs)):
    if ba_local_implicit_dofs[i] == 1:
        gfuim.vec.data[i] = 1
        
scene1 = Draw(gfuim.components[0], mesh)
scene2 = Draw(gfuim.components[1], mesh)

In [ ]:
gfp.Set( exp(-400*(x**2+y**2+z**2)))
gfu.vec[:] = 0
gfp_halfstep = gfp.vec.CreateVector()
gfuold = gfu.vec.CreateVector()
res = gfu.vec.CreateVector()

if dim == 2:
    scene = Draw (gfp, order=3, deformation=True);
else:
    scene = Draw (gftr, draw_vol=False, order=3);
    
t = 0
tend = 2
cnt = 0
loop = tqdm([i*dt for i in range(int(tend/dt))])
with TaskManager():#pajetrace=10**8):
    for i in loop:
        
        gfp_halfstep.data = gfp.vec - dt/2 * invmassp @ B.T * gfu.vec
        
        res.data = dt * B_e * gfp_halfstep + dt/2 * B_i * (gfp_halfstep + gfp.vec) + massu * gfu.vec
        gfuold.data = invmassu * res 
        gfu.vec.data = gfuold
        gfu.vec.data += invmstar * (res - mstarloc * gfuold)
        gfp.vec.data = gfp_halfstep - dt/2 * invmassp @ B.T * gfu.vec
        cnt = cnt+1
        if cnt%10 == 0:
            if dim == 3:
                gftr.vec.data = traceop * gfp.vec
        scene.Redraw()
        loop.set_description(f'Time step {i/dt:.0f}')
        loop.set_postfix_str(f'Time {i:.4f} seconds')

In [ ]:
import scipy.sparse as sp
import matplotlib.pylab as plt
rows,cols,vals = A.mat.COO()
a = sp.csr_matrix((vals,(rows,cols)))
plt.spy(a)
plt.show()